# 유튜브 영상에서 오디오 추출하기

## ffmpeg 설치
* https://ffmpeg.org/에서 ffmpeg 다운로드(https://github.com/BtbN/FFmpeg-Builds/releases)
* 압축 풀어 설치
* path 환경변수에 bin 폴더 추가
* IDE 재시작

In [ ]:
import yt_dlp
import os
import shutil

def download_youtube_mp3(video_id: str = 'gqXEEydkbh4'):
    # FFmpeg 설치 여부 확인
    if shutil.which("ffmpeg") is None:
        print("오류: FFmpeg이 설치되어 있지 않습니다. 설치 후 다시 시도하세요.")
        return None

    url = f"https://www.youtube.com/watch?v={video_id}"
    output_dir = "downloads"
    os.makedirs(output_dir, exist_ok=True)  # 다운로드 폴더 생성

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f"{output_dir}/%(id)s.%(ext)s",
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            file_name = f"{info['id']}.mp3"
            file_path = os.path.join(output_dir, file_name)

        print(f"다운로드 완료! 파일 경로: {file_path}")
        return file_path
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

# 테스트 실행 gqXEEydkbh4
audio_path = download_youtube_mp3('Phajxmf9j9I')
print(audio_path)

# 파일 자르기

In [ ]:
import ffmpeg
import os

def trim_audio_file(file_path:str, max_size:int=24*1024*1024):
    file_size = os.path.getsize(file_path)  # 바이트 단위
    output_path = file_path
    if file_size >= max_size:
        # ffmpeg로 오디오 비트레이트 가져오기
        probe = ffmpeg.probe(file_path)
        bitrate = int(probe['format']['bit_rate'])  # 비트레이트 (bps)

        # 최대 24MB로 자를 수 있는 재생 시간 계산
        max_size = 24*1024*1024 # 25MB 보다 작게
        max_duration = max_size * 8 / bitrate  # 초 단위로 변환

        # 출력 파일 경로 설정
        output_path = file_path.replace(".mp3", "_trimmed.mp3")

        # ffmpeg로 파일 자르기
        ffmpeg.input(file_path, ss=0, t=max_duration).output(output_path, codec="copy").run(overwrite_output=True)
    return output_path

trimmed_audio_path = trim_audio_file(audio_path)

# API키 설정하기

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key = api_key
)

# 오디오 파일에서 텍스트 추출하기

In [ ]:
def transcribe_audio(audio_path):
    audio_file= open(audio_path, "rb")
    response = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="text"
    )
    return response

# 텍스트 변환 실행
transcribed_text = transcribe_audio(trimmed_audio_path)
print("변환된 텍스트:\n", transcribed_text)

# 텍스트 내용 요약하기

In [ ]:
def summarize_text(text):
     response = client.chat.completions.create(
          model="gpt-4o-mini",
          temperature=0,
          messages=[
               { "role": "system", "content": "주어진 텍스트를 간결하고 명확하게 요약해 주세요." },
               { "role": "user", "content": text }
          ],
     )
     return response.choices[0].message.content

# 요약 실행
summary = summarize_text(transcribed_text)
print("요약된 내용:\n", summary)